In [1]:
import re
import nltk
import sklearn
import numpy as n
import pandas as pd
from nltk.stem import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#### Importing Datasets Primary and Secondary Dataset


In [2]:
#Primary Twitter.topic
df = pd.read_csv(r'primary_final\twitter_topic_lockdowns.csv')
df1 = pd.read_csv(r'primary_final\twitter_topic_masking_and_distancing.csv')
df2 = pd.read_csv(r'primary_final\twitter_topic_vaccination.csv')
#Secondary Change.org.topic
df_s = pd.read_csv(r'secondary_final\change.org_topic_lockdowns.csv')
df1_s = pd.read_csv(r'secondary_final\change.org_topic_masking_and_distancing.csv')
df2_s = pd.read_csv(r'secondary_final\change.org_topic_vaccination.csv')

In [3]:
df.count()

text           1200
Final Label    1200
dtype: int64

In [4]:
#Primary Lockdown
df.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,False
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,False
2,4 of the vaccines Jared bought are expected to...,False
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,False


In [5]:
df.describe()

,text,Final Label
count,1200,1200
unique,1195,2
top,STUDIES SHOW MORPHINE MILLIGRAM EQUIVALENT (MM...,False
freq,4,1145


In [6]:
#Primary Masking and Distancing
df1.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,False
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,False
2,4 of the vaccines Jared bought are expected to...,False
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,True


In [7]:
df1.describe()

,text,Final Label
count,1200,1200
unique,1195,2
top,STUDIES SHOW MORPHINE MILLIGRAM EQUIVALENT (MM...,False
freq,4,957


In [8]:
#Primary Vaccination
df2.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,True
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,True
2,4 of the vaccines Jared bought are expected to...,True
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,True


In [9]:
df2.describe()

,text,Final Label
count,1200,1200
unique,1195,2
top,STUDIES SHOW MORPHINE MILLIGRAM EQUIVALENT (MM...,True
freq,4,701


### Secondary Dataset

In [10]:
#Secondary Lockdown
df_s.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [11]:
df_s.describe()

,text,Final Label
count,1500,1500
unique,1500,2
top,Force Big Chain Supermarkets/Takeaways to Drop...,False
freq,1,1071


In [12]:
#Secondary Masking and Distancing
df1_s.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [13]:
df1_s.describe()

,text,Final Label
count,1500,1500
unique,1500,2
top,Force Big Chain Supermarkets/Takeaways to Drop...,False
freq,1,1429


In [14]:
#Secondary Vaccinatioon
df2_s.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [15]:
df2_s.describe()

,text,Final Label
count,1500,1500
unique,1500,2
top,Force Big Chain Supermarkets/Takeaways to Drop...,False
freq,1,1429


### Checking the count of Positive Classes

In [16]:
print("Primary Lockdowns:\n", df['Final Label'].value_counts(True)*100)
print("Primary Masking and Distancing:\n",df1['Final Label'].value_counts(True)*100)
print("Primary Vaccinations:\n",df2['Final Label'].value_counts(True)*100)
print("Secondary Lockdowns:\n",df_s['Final Label'].value_counts(True)*100)
print("Secondary Masking and Distancing:\n",df1_s['Final Label'].value_counts(True)*100)
print("Secondary Vaccinations:\n",df2_s['Final Label'].value_counts(True)*100)

Primary Lockdowns:
 False    95.416667
True      4.583333
Name: Final Label, dtype: float64
Primary Masking and Distancing:
 False    79.75
True     20.25
Name: Final Label, dtype: float64
Primary Vaccinations:
 True     58.416667
False    41.583333
Name: Final Label, dtype: float64
Secondary Lockdowns:
 False    71.4
True     28.6
Name: Final Label, dtype: float64
Secondary Masking and Distancing:
 False    95.266667
True      4.733333
Name: Final Label, dtype: float64
Secondary Vaccinations:
 False    95.266667
True      4.733333
Name: Final Label, dtype: float64


### Explanation to select right Primary dataset for training and modelling
#### From above we can see that in dataset twitter_topic_lockdowns and dataset twitter_topic_masking_and_distancing have low numbers of positive examples, as shown by frequency which represent number of False in datasets is 95% and 79% respectively. <br> 


### Therefore, we will select twitter_topic_vaccination.csv for Data partitioning and  Baseline modelling. Also, change.org_topic_vaccination from Secondary Dataset and designated 100% of SECONDARY dataset as the Test (SECONDARY) dataset. 

In [17]:
df_primary = df2.copy()
df_primary.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,True
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,True
2,4 of the vaccines Jared bought are expected to...,True
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,True


In [18]:
df_secondary = df2_s.copy()
df_secondary.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [19]:
df_secondary.isnull().sum()

text           0
Final Label    0
dtype: int64

#### Model 1 
#### Before Feature Engineering
1. Training dataset = Primary Data 70%
2. Test Dataset = Primary Data 30%


In [20]:
print(df_primary)

                                                   text  Final Label
0     Putin After Announcing #CovidVaccine #Russian ...         True
1     Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...         True
2     4 of the vaccines Jared bought are expected to...         True
3     One day you will realize CDC Guidelines magica...        False
4     Im far from lying.  Current CDC guidelines is ...         True
...                                                 ...          ...
1195  .Sprint To Develop A #COVIDVaccine – // https:...         True
1196  Here in the U.S. some localities have brought ...        False
1197  Sanitizer &amp; Mask Manufacturers After Russi...         True
1198  We are following all CDC guidelines through a ...        False
1199  People be losing hopes on covid vaccine like i...         True

[1200 rows x 2 columns]


In [21]:
X_prim = df_primary.drop(["Final Label"],axis = 1)
y_prim = df_primary["Final Label"]

X_sec = df_secondary.drop(["Final Label"],axis = 1)
y_sec = df_secondary["Final Label"]

In [22]:
#Splitting the Dataset into Primary Train and Primary Test
X_prim_train,X_prim_test,y_prim_train,y_prim_test = train_test_split(X_prim,y_prim,test_size=0.3,random_state = 42)

In [23]:
X_prim_train.reset_index(inplace=True)
X_prim_test.reset_index(inplace=True)

X_prim_train = X_prim_train.drop("index",axis = 1)
X_prim_test = X_prim_test.drop("index",axis = 1)

In [24]:
tfid = TfidfVectorizer()

final_x_prim_train = tfid.fit_transform(X_prim_train.text).toarray()
df = pd.DataFrame(final_x_prim_train)  
final_x_prim_train = X_prim_train.join(df) 
print("Primary train with Feature Engineering:",final_x_prim_train.shape)


final_x_prim_test = tfid.transform(X_prim_test.text).toarray()
df1 = pd.DataFrame(final_x_prim_test)  
final_x_prim_test = X_prim_test.join(df1) 
print("Primary test with Feature Engineering:",final_x_prim_test.shape)


Primary train with Feature Engineering: (840, 4390)
Primary test with Feature Engineering: (360, 4390)


In [25]:
final_x_prim_train = final_x_prim_train.drop("text",axis = 1)
final_x_prim_test = final_x_prim_test.drop("text",axis = 1)

### Using Random Forest Classifier on model 1

In [26]:
from sklearn.naive_bayes import GaussianNB
#clf = GaussianNB()
clf = RandomForestClassifier(random_state = 42)
clf.fit(final_x_prim_train, y_prim_train)
y_pred_prim = clf.predict(final_x_prim_test)

print("The Classification Report :\n",classification_report(y_prim_test,y_pred_prim))
print("\nConfusion Matrix : \n",confusion_matrix(y_prim_test,y_pred_prim))
print("\nAccuracy in Primary Set using Random Forest:",round((accuracy_score(y_prim_test,y_pred_prim))*100,2))

The Classification Report :
               precision    recall  f1-score   support

       False       0.84      0.78      0.81       139
        True       0.87      0.91      0.89       221

    accuracy                           0.86       360
   macro avg       0.86      0.85      0.85       360
weighted avg       0.86      0.86      0.86       360


Confusion Matrix : 
 [[109  30]
 [ 20 201]]

Accuracy in Primary Set using Random Forest: 86.11


#### Model 2
#### Before Feature Engineering
1. Training dataset = Primary Data 70%
2. Test Dataset = Secondary Data 100%


In [27]:
df_primary.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,True
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,True
2,4 of the vaccines Jared bought are expected to...,True
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,True


In [28]:
df_secondary.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [29]:
final_x_sec_test = tfid.transform(X_sec.text).toarray()
df2 = pd.DataFrame(final_x_sec_test)  
final_x_sec_test = X_sec.join(df2) 
print("Primary test with Feature Engineering:",final_x_sec_test.shape)

Primary test with Feature Engineering: (1500, 4390)


In [30]:
final_x_sec_test = final_x_sec_test.drop("text",axis = 1)

### Using Random Forest Classifier on model 2

In [31]:
clf = RandomForestClassifier(random_state = 42)
clf.fit(final_x_prim_train, y_prim_train)
y_pred_sec = clf.predict(final_x_sec_test)

print("Classification Report: \n",classification_report(y_sec,y_pred_sec))
print("\nConfusion Matrix : \n",confusion_matrix(y_sec,y_pred_sec))
print("\nAccuracy in Secondary Set using Random Forest Classifier :",round((accuracy_score(y_sec,y_pred_sec))*100,2))

Classification Report: 
               precision    recall  f1-score   support

       False       0.96      0.32      0.48      1429
        True       0.05      0.76      0.10        71

    accuracy                           0.34      1500
   macro avg       0.51      0.54      0.29      1500
weighted avg       0.92      0.34      0.46      1500


Confusion Matrix : 
 [[457 972]
 [ 17  54]]

Accuracy in Secondary Set using Random Forest Classifier : 34.07


From above it is clear that using Random Forest Classifier we will get 

1. Accuracy on Primary DataSet(Vaccination) Model 1 as ~86.11%

2. Accuracy on Secondary DataSet(Vaccination) Model 2 as ~34.07%


### 4. Feature Engineering 

In [32]:
print('Text count Primary:',df_primary['text'].count())
print('\nText count Secondary:',df_secondary['text'].count())

Text count Primary: 1200

Text count Secondary: 1500


In [33]:
#Number of Characters
def count_chars(text):
    return len(text)

#Number of Words
def count_words(text):
    return len(text.split())

def count_sent(text):
    return len(nltk.sent_tokenize(text))

#Number of Capital Characters
def count_capital_chars(text):
    count=0
    for i in text:
        if i.isupper():
            count+=1
    return count

def count_unique_words(text):
    return len(set(text.split()))

#Count of mentions
def count_mentions(text):
    x = re.findall(r'(@w[A-Za-z0-9]*)', text)
    return len(x)



In [34]:
def cleaning_pipe(sent):
    #Removing Stopwords and Punctuations
    data = re.sub('[^a-zA-Z]', ' ',sent)
    data = data.lower()
    data = data.split()
    data = [word for word in data if not word in set(stopwords.words('english'))]
    data = ' '.join(data)
    
    #Performing Stemming using PortarStemmer
    stemmer = PorterStemmer()
    words = word_tokenize(data)
    words = [ stemmer.stem(word) for word in words ]
    data = ' '.join(words)
    return data

In [35]:
df_primary['char_count'] = df_primary["text"].apply(lambda x:count_chars(x))
df_primary['word_count'] = df_primary["text"].apply(lambda x:count_words(x))
df_primary['sent_count'] = df_primary["text"].apply(lambda x:count_sent(x))
df_primary['avg_wordlength'] = df_primary['char_count']/df_primary['word_count']
df_primary['avg_sentlength'] = df_primary['word_count']/df_primary['sent_count']
df_primary['unique_word_count'] = df_primary["text"].apply(lambda x:count_unique_words(x))
df_primary['unique_vs_words'] = df_primary['unique_word_count']/df_primary['word_count']
df_primary["text"] = df_primary["text"].apply(cleaning_pipe)

In [36]:
df_secondary['char_count'] = df_secondary["text"].apply(lambda x:count_chars(x))
df_secondary['word_count'] = df_secondary["text"].apply(lambda x:count_words(x))
df_secondary['sent_count'] = df_secondary["text"].apply(lambda x:count_sent(x))
df_secondary['avg_wordlength'] = df_secondary['char_count']/df_secondary['word_count']
df_secondary['avg_sentlength'] = df_secondary['word_count']/df_secondary['sent_count']
df_secondary['unique_word_count'] = df_secondary["text"].apply(lambda x:count_unique_words(x))
df_secondary['unique_vs_words'] = df_secondary['unique_word_count']/df_secondary['word_count']
df_secondary["text"] = df_secondary["text"].apply(cleaning_pipe)

In [37]:
df_primary.isnull().sum()

text                 0
Final Label          0
char_count           0
word_count           0
sent_count           0
avg_wordlength       0
avg_sentlength       0
unique_word_count    0
unique_vs_words      0
dtype: int64

In [38]:
df_secondary.isnull().sum()

text                 0
Final Label          0
char_count           0
word_count           0
sent_count           0
avg_wordlength       0
avg_sentlength       0
unique_word_count    0
unique_vs_words      0
dtype: int64

In [39]:
df_primary.head()

,text,Final Label,char_count,word_count,sent_count,avg_wordlength,avg_sentlength,unique_word_count,unique_vs_words
0,putin announc covidvaccin russian http co rndf...,True,69,6,1,11.500000,6.000000,6,1.000000
1,courtesi wa whatsapp covid covidvaccin http co...,True,68,6,2,11.333333,3.000000,6,1.000000
2,vaccin jare bought expect fail trump refus joi...,True,113,18,1,6.277778,18.000000,16,0.888889
3,one day realiz cdc guidelin magic align teache...,False,86,13,1,6.615385,13.000000,13,1.000000
4,im far lie current cdc guidelin wear mask indo...,True,113,20,3,5.650000,6.666667,20,1.000000


In [40]:
df_secondary.head()

,text,Final Label,char_count,word_count,sent_count,avg_wordlength,avg_sentlength,unique_word_count,unique_vs_words
0,languag educ time covid,False,42,7,1,6.000000,7.0,7,1.000000
1,covid test kit,False,18,3,1,6.000000,3.0,3,1.000000
2,covid prison,False,18,4,1,4.500000,4.0,4,1.000000
3,get wale home,False,14,3,1,4.666667,3.0,3,1.000000
4,make pass fail avail mississippi state student...,False,88,12,1,7.333333,12.0,11,0.916667


In [41]:
X_prim = df_primary.drop(["Final Label"],axis = 1)
y_prim = df_primary["Final Label"]

X_sec = df_secondary.drop(["Final Label"],axis = 1)
y_sec = df_secondary["Final Label"]

### Model 1 after Feature Engineering

In [42]:
#Splitting the Dataset into Primary Train and Primary Test
X_prim_train,X_prim_test,y_prim_train,y_prim_test = train_test_split(X_prim,y_prim,test_size=0.3,random_state = 42)

In [43]:
X_prim_train.reset_index(inplace=True)
X_prim_test.reset_index(inplace=True)

X_prim_train = X_prim_train.drop("index",axis = 1)
X_prim_test = X_prim_test.drop("index",axis = 1)

### Creating the Dataframe for the TF-IDF Vectorizer

In [44]:
tfif = TfidfVectorizer()

final_x_prim_train = tfid.fit_transform(X_prim_train.text).toarray()
df = pd.DataFrame(final_x_prim_train)  
final_x_prim_train = X_prim_train.join(df) 
print("Primary train with feature Engineering :",final_x_prim_train.shape)


final_x_prim_test = tfid.transform(X_prim_test.text).toarray()
df1 = pd.DataFrame(final_x_prim_test)  
final_x_prim_test = X_prim_test.join(df1) 
print("Primary test with feature Engineering :",final_x_prim_test.shape)

Primary train with feature Engineering : (840, 3666)
Primary test with feature Engineering : (360, 3666)


In [45]:
final_x_prim_train = final_x_prim_train.drop("text",axis = 1)
final_x_prim_test = final_x_prim_test.drop("text",axis = 1)

In [46]:
clf = RandomForestClassifier(random_state = 42)
clf.fit(final_x_prim_train, y_prim_train)
y_pred_prim = clf.predict(final_x_prim_test)

print("Classification Report :\n",classification_report(y_prim_test,y_pred_prim))
print("\nConfusion Matrix : \n",confusion_matrix(y_prim_test,y_pred_prim))
print("\nAccuracy in Primary Set using Random Forest Classfier:",round((accuracy_score(y_prim_test,y_pred_prim))*100,2))

Classification Report :
               precision    recall  f1-score   support

       False       0.90      0.81      0.85       139
        True       0.89      0.95      0.91       221

    accuracy                           0.89       360
   macro avg       0.89      0.88      0.88       360
weighted avg       0.89      0.89      0.89       360


Confusion Matrix : 
 [[112  27]
 [ 12 209]]

Accuracy in Primary Set using Random Forest Classfier: 89.17


### Model 2 after Feature Engineering

In [47]:
final_x_sec_test = tfid.transform(X_sec.text).toarray()
df2 = pd.DataFrame(final_x_sec_test)  
final_x_sec_test = X_sec.join(df2) 
print("The size of Primary test with feature Engineering is:",final_x_sec_test.shape)

The size of Primary test with feature Engineering is: (1500, 3666)


In [48]:
final_x_sec_test = final_x_sec_test.drop("text",axis = 1)

### Model using Random Forest Classifier

In [49]:
clf = RandomForestClassifier(random_state = 42)
clf.fit(final_x_prim_train, y_prim_train)
y_pred_sec = clf.predict(final_x_sec_test)

print("The Classification Report: \n",classification_report(y_sec,y_pred_sec))
print("\nConfusion Matrix : \n",confusion_matrix(y_sec,y_pred_sec))
print("\nThe Accuracy in Secondary Set using Random Forest Classifier is:",round((accuracy_score(y_sec,y_pred_sec))*100,2))

The Classification Report: 
               precision    recall  f1-score   support

       False       0.96      0.47      0.63      1429
        True       0.05      0.59      0.10        71

    accuracy                           0.48      1500
   macro avg       0.51      0.53      0.37      1500
weighted avg       0.92      0.48      0.61      1500


Confusion Matrix : 
 [[677 752]
 [ 29  42]]

The Accuracy in Secondary Set using Random Forest Classifier is: 47.93


From above it is clear that using RandomForest we will get 

1. Accuracy on Primary DataSet(Vaccination) Model 1 as ~89.17%

2. Accuracy on Secondary DataSet(Vaccination) Model 2 as ~47.93%

